In [ ]:
#!pip install ultralytics
from ultralytics import YOLO
import torch.nn as nn
import zipfile
import tensorflow as tf
import shutil
from google.colab import files
import pandas as pd
import os, time
import numpy as np

#Replacing the classifier head with one for our dataset
model = YOLO("yolo11s-cls.pt")
model.info()
print(model.model)

model.model.model[10].linear = nn.Linear(
    in_features=1280,
    out_features=8,
    bias=True
)
#Freeze rest of model
for name, param in model.model.named_parameters():
    param.requires_grad = False
#unfreeze classifier head
for param in model.model.model[10].linear.parameters():
    param.requires_grad = True

YOLO11s-cls summary: 86 layers, 6,724,008 parameters, 0 gradients, 13.2 GFLOPs
ClassificationModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, af

In [ ]:
zip_path = '/content/greyscale_downsampled_split.zip'
extract_path = '/content/greyscale_downsampled_split'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [19]:
start_time = time.time()

results = model.train(data=r"/content/greyscale_downsampled_split/greyscale_downsampled_split", epochs=100, patience=5, lr0=0.001, imgsz=224, device="cuda:0")
end_time = time.time()
training_duration = end_time - start_time
print(f"Training time: {training_duration:.2f} seconds")

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/greyscale_downsampled_split/greyscale_downsampled_split, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience

In [20]:
metrics = model.val(data=r"/content/greyscale_downsampled_split/greyscale_downsampled_split/test")
print(metrics)

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11s-cls summary (fused): 47 layers, 5,444,376 parameters, 0 gradients, 12.0 GFLOPs
WARNING ⚠️ Dataset 'split=train' not found at /content/greyscale_downsampled_split/greyscale_downsampled_split/test/train
Found 4317 images in subdirectories. Attempting to split...
Splitting /content/greyscale_downsampled_split/greyscale_downsampled_split/test (8 classes, 4317 images) into 80% train, 20% val...
Split complete in /content/greyscale_downsampled_split/greyscale_downsampled_split/test_split ✅
train: /content/greyscale_downsampled_split/greyscale_downsampled_split/test_split/train... found 3489 images in 8 classes ✅ 
val: /content/greyscale_downsampled_split/greyscale_downsampled_split/test_split/val... found 902 images in 8 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 477.5±121.2 MB/s, size: 25.3 KB)
val: Scanning /content/greyscale_downsampled_split/greyscale_downsampled_sp

In [21]:
#Grey Model
model.export(format="saved_model", keras=True)

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)

PyTorch: starting from '/content/runs/classify/train4/weights/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 8) (10.5 MB)

TensorFlow SavedModel: starting export with tensorflow 2.19.0...

ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.74...
ONNX: export success ✅ 0.8s, saved as '/content/runs/classify/train4/weights/best.onnx' (20.8 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.28.3...
Saved artifact at '/content/runs/classify/train4/weights/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 224, 224, 3), dtype=tf.float32, name='images')
Output Type:
  TensorSpec(shape=(1, 8), dtype=tf.float32, name=None)
Captures:
  134133232150736: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  134133232155536: TensorSpec(shape=(3, 3, 3, 32)

'/content/runs/classify/train4/weights/best_saved_model'

In [ ]:
loaded_model = tf.keras.models.load_model(r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\best_saved_model (1)")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('model_yolov11_quantized_grey.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
loaded_model = tf.keras.models.load_model(r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\best_saved_model (1)")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()
with open('model_yolov11_grey.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:

# Zip the folder
shutil.make_archive("/content/runs/classify/train4/weights/best_saved_model", 'zip', "/content/runs/classify/train4/weights/best_saved_model")

# Download the zip
files.download("/content/runs/classify/train4/weights/best_saved_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
model = YOLO("yolo11s-cls.pt")
model.info()
print(model.model)

model.model.model[10].linear = nn.Linear(
    in_features=1280,
    out_features=8,
    bias=True
)
#Freeze rest of model
for name, param in model.model.named_parameters():
    param.requires_grad = False
#unfreeze classifier head
for param in model.model.model[10].linear.parameters():
    param.requires_grad = True

YOLO11s-cls summary: 86 layers, 6,724,008 parameters, 0 gradients, 13.2 GFLOPs
ClassificationModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, af

In [ ]:
zip_path = '/content/original_downsampled_split.zip'
extract_path = '/content/original_downsampled_split'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [13]:
start_time = time.time()
#Baseline Results
results = model.train(data=r"/content/original_downsampled_split/original_downsampled_split", epochs=100, patience=5, lr0=0.001, imgsz=224, device="cuda:0")

end_time = time.time()
training_duration = end_time - start_time
print(f"Training time: {training_duration:.2f} seconds")

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/original_downsampled_split/original_downsampled_split, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=5

In [15]:
metrics = model.val(data=r"/content/original_downsampled_split/original_downsampled_split/test")
print(metrics)

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
WARNING ⚠️ Dataset 'split=train' not found at /content/original_downsampled_split/original_downsampled_split/test/train
Found 4317 images in subdirectories. Attempting to split...
Splitting /content/original_downsampled_split/original_downsampled_split/test (8 classes, 4317 images) into 80% train, 20% val...
Split complete in /content/original_downsampled_split/original_downsampled_split/test_split ✅
train: /content/original_downsampled_split/original_downsampled_split/test_split/train... found 3452 images in 8 classes ✅ 
val: /content/original_downsampled_split/original_downsampled_split/test_split/val... found 865 images in 8 classes ✅ 
test: None...
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 860.2±305.6 MB/s, size: 29.8 KB)
val: Scanning /content/original_downsampled_split/original_downsampled_split/test_split/val... 865 images, 0 corrupt: 100% ━━━━━━━━━━━━ 865/865 3.0Kit/s 0.3s
val: New cache

In [16]:
model.export(format="saved_model", keras=True)

Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)

PyTorch: starting from '/content/runs/classify/train3/weights/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 8) (10.5 MB)

TensorFlow SavedModel: starting export with tensorflow 2.19.0...

ONNX: starting export with onnx 1.19.1 opset 22...
ONNX: slimming with onnxslim 0.1.74...
ONNX: export success ✅ 0.8s, saved as '/content/runs/classify/train3/weights/best.onnx' (20.8 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.28.3...
Saved artifact at '/content/runs/classify/train3/weights/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 224, 224, 3), dtype=tf.float32, name='images')
Output Type:
  TensorSpec(shape=(1, 8), dtype=tf.float32, name=None)
Captures:
  134134925218192: TensorSpec(shape=(4, 2), dtype=tf.int32, name=None)
  134134925220304: TensorSpec(shape=(3, 3, 3, 32)

'/content/runs/classify/train3/weights/best_saved_model'

In [ ]:
# Zip the folder
shutil.make_archive("/content/runs/classify/train3/weights/best_saved_model", 'zip', "/content/runs/classify/train3/weights/best_saved_model")

# Download the zip
files.download("/content/runs/classify/train3/weights/best_saved_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#YOLOv11 quantized trained on grey images
loaded_model = tf.keras.models.load_model(r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\Final_Project_Models\Create_Models\YOLOv11\Grey\best_saved_model")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('model_yolov11_quantized_grey.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
#YOLOv11 trained on grey images
loaded_model = tf.keras.models.load_model(r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\Final_Project_Models\Create_Models\YOLOv11\Grey\best_saved_model")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()
with open('model_yolov11_grey.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
#YOLOv11 quantized trained on original images
loaded_model = tf.keras.models.load_model(r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\Final_Project_Models\Create_Models\YOLOv11\Original\best_saved_model")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with open('model_yolov11_quantized_original.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
#YOLOv11 quantized trained on original images
loaded_model = tf.keras.models.load_model(r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\Final_Project_Models\Create_Models\YOLOv11\Original\best_saved_model")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()
with open('model_yolov11_original.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
#Load the test dataset
data_dir = r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\Final_Project_Models\Create_Models\data\north_american_predators\greyscale_downsampled_split\test"
dataset_test = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  seed=123,
  image_size=(224, 224),
  batch_size=64)

Found 4317 files belonging to 8 classes.


In [ ]:
def evaluate_model_tflite(interpreter, dataset):
    #List of dictionaries
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    correct = 0
    total = 0

    for images, labels in dataset:
        input_type = input_details[0]['dtype']
        # Convert the batch to the correct type
        images = images.numpy().astype(input_type)
        labels = labels.numpy()
        
        for i in range(images.shape[0]):
            test_image = np.expand_dims(images[i], axis=0)
            #From Tensorflow API Docs
            interpreter.set_tensor(input_details[0]['index'], test_image)
            interpreter.invoke()
            output = interpreter.get_tensor(output_details[0]['index'])
            #Gets the most prominent class
            pred = np.argmax(output)
            if pred == labels[i]:
                correct += 1
            total += 1

    accuracy = correct / total
    return accuracy

In [ ]:
#YOLOv11 trained on original images
interpreter = tf.lite.Interpreter(model_path=r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\Final_Project_Models\Create_Models\YOLOv11\Original\model_yolov11_original.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)        
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize(r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\Final_Project_Models\Create_Models\YOLOv11\Original\model_yolov11_original.tflite") / float(2**20))

TFLite accuracy: 0.41649293490850126
TFlite model in Mb: 20.802791595458984


In [ ]:
#YOLOv11 quantized trained on original images
interpreter = tf.lite.Interpreter(model_path=r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\Final_Project_Models\Create_Models\YOLOv11\Original\model_yolov11_quantized_original.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)        
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize(r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\Final_Project_Models\Create_Models\YOLOv11\Original\model_yolov11_quantized_original.tflite") / float(2**20))

TFLite accuracy: 0.41649293490850126
TFlite model in Mb: 5.356437683105469


In [ ]:
#YOLOv11 trained on grey images
interpreter = tf.lite.Interpreter(model_path=r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\Final_Project_Models\Create_Models\YOLOv11\Grey\model_yolov11_grey.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)        
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize(r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\Final_Project_Models\Create_Models\YOLOv11\Grey\model_yolov11_grey.tflite") / float(2**20))

TFLite accuracy: 0.4157980078758397
TFlite model in Mb: 20.803260803222656


In [ ]:
#YOLOv11 quantized trained on grey images
interpreter = tf.lite.Interpreter(model_path=r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\Final_Project_Models\Create_Models\YOLOv11\Grey\model_yolov11_quantized_grey.tflite")
interpreter.allocate_tensors()
acc = evaluate_model_tflite(interpreter, dataset_test)        
print("TFLite accuracy:", acc)
print("TFlite model in Mb:", os.path.getsize(r"C:\Graduate_School\Fall_2025\ECE528\Final_Project\Final_Project_Models\Create_Models\YOLOv11\Grey\model_yolov11_quantized_grey.tflite") / float(2**20))

TFLite accuracy: 0.41649293490850126
TFlite model in Mb: 5.356903076171875
